# Downloading the data

This notebook is used to download the ERA5 data from weatherbench2.


In [1]:
import xarray as xr
import os
import fsspec
import numpy as np
import pandas as pd
import jinja2

In [2]:
#!gcloud auth login
#!gsutil -m cp -r "gs://weatherbench2/datasets/era5/1959-2022-6h-64x32_equiangular_conservative.zarr" ./ERA5_data

/bin/bash: line 1: gcloud: command not found


In [2]:

# Define the relative path to the file
relative_path = "ERA5_data/1959-2022-6h-64x32_equiangular_conservative.zarr"

# Get the absolute path by joining the current directory with the relative path
absolute_path = os.path.join(os.path.dirname(os.getcwd()), relative_path)
print(absolute_path)


/home/jupyter-lukas/Masters-Thesis/ERA5_data/1959-2022-6h-64x32_equiangular_conservative.zarr


In [9]:

# Define the path to the Zarr file
file_path = "Masters-Thesis/ERA5_data/1959-2022-6h-64x32_equiangular_conservative.zarr"

# Open the Zarr file using xarray
obs_data = xr.open_zarr(absolute_path)
obs_data


<xarray.Dataset> Size: 63GB
Dimensions:                       (batch: 1, time: 92044, lon: 64, lat: 32,
                                   level: 13)
Coordinates:
    datetime                      (time) datetime64[ns] 736kB dask.array<chunksize=(23011,), meta=np.ndarray>
  * lat                           (lat) float64 256B -87.19 -81.56 ... 87.19
  * level                         (level) int64 104B 50 100 150 ... 850 925 1000
  * lon                           (lon) float64 512B 0.0 5.625 ... 348.8 354.4
  * time                          (time) datetime64[ns] 736kB 1959-01-01 ... ...
Dimensions without coordinates: batch
Data variables: (12/14)
    10m_u_component_of_wind       (batch, time, lon, lat) float32 754MB dask.array<chunksize=(1, 100, 64, 32), meta=np.ndarray>
    10m_v_component_of_wind       (batch, time, lon, lat) float32 754MB dask.array<chunksize=(1, 100, 64, 32), meta=np.ndarray>
    2m_temperature                (batch, time, lon, lat) float32 754MB dask.array<chunksize=(1, 100, 64, 32), meta=np.ndarray>
    geopotential                  (batch, time, level, lon, lat) float32 10GB dask.array<chunksize=(1, 100, 13, 64, 32), meta=np.ndarray>
    geopotential_at_surface       (batch, lon, lat) float32 8kB dask.array<chunksize=(1, 64, 32), meta=np.ndarray>
    land_sea_mask                 (batch, lon, lat) float32 8kB dask.array<chunksize=(1, 64, 32), meta=np.ndarray>
    ...                            ...
    temperature                   (batch, time, level, lon, lat) float32 10GB dask.array<chunksize=(1, 100, 13, 64, 32), meta=np.ndarray>
    toa_incident_solar_radiation  (batch, time, lon, lat) float32 754MB dask.array<chunksize=(1, 100, 64, 32), meta=np.ndarray>
    total_precipitation_6hr       (batch, time, lon, lat) float32 754MB dask.array<chunksize=(1, 100, 64, 32), meta=np.ndarray>
    u_component_of_wind           (batch, time, level, lon, lat) float32 10GB dask.array<chunksize=(1, 100, 13, 64, 32), meta=np.ndarray>
    v_component_of_wind           (batch, time, level, lon, lat) float32 10GB dask.array<chunksize=(1, 100, 13, 64, 32), meta=np.ndarray>
    vertical_velocity             (batch, time, level, lon, lat) float32 10GB dask.array<chunksize=(1, 100, 13, 64, 32), meta=np.ndarray>

In [4]:
# prune the dataset of unnecessary variables.
for x in obs_data.data_vars:
    if x not in ['2m_temperature','mean_sea_level_pressure','10m_v_component_of_wind',
        '10m_u_component_of_wind','total_precipitation_6hr','toa_incident_solar_radiation',
        'temperature','geopotential','geopotential_at_surface','land_sea_mask',
        'u_component_of_wind','v_component_of_wind','vertical_velocity','specific_humidity']:
        obs_data = obs_data.drop_vars(x)

obs_data

<xarray.Dataset> Size: 63GB
Dimensions:                       (time: 92044, longitude: 64, latitude: 32,
                                   level: 13)
Coordinates:
  * latitude                      (latitude) float64 256B -87.19 ... 87.19
  * level                         (level) int64 104B 50 100 150 ... 850 925 1000
  * longitude                     (longitude) float64 512B 0.0 5.625 ... 354.4
  * time                          (time) datetime64[ns] 736kB 1959-01-01 ... ...
Data variables: (12/14)
    10m_u_component_of_wind       (time, longitude, latitude) float32 754MB dask.array<chunksize=(100, 64, 32), meta=np.ndarray>
    10m_v_component_of_wind       (time, longitude, latitude) float32 754MB dask.array<chunksize=(100, 64, 32), meta=np.ndarray>
    2m_temperature                (time, longitude, latitude) float32 754MB dask.array<chunksize=(100, 64, 32), meta=np.ndarray>
    geopotential                  (time, level, longitude, latitude) float32 10GB dask.array<chunksize=(100, 13, 64, 32), meta=np.ndarray>
    geopotential_at_surface       (longitude, latitude) float32 8kB dask.array<chunksize=(64, 32), meta=np.ndarray>
    land_sea_mask                 (longitude, latitude) float32 8kB dask.array<chunksize=(64, 32), meta=np.ndarray>
    ...                            ...
    temperature                   (time, level, longitude, latitude) float32 10GB dask.array<chunksize=(100, 13, 64, 32), meta=np.ndarray>
    toa_incident_solar_radiation  (time, longitude, latitude) float32 754MB dask.array<chunksize=(100, 64, 32), meta=np.ndarray>
    total_precipitation_6hr       (time, longitude, latitude) float32 754MB dask.array<chunksize=(100, 64, 32), meta=np.ndarray>
    u_component_of_wind           (time, level, longitude, latitude) float32 10GB dask.array<chunksize=(100, 13, 64, 32), meta=np.ndarray>
    v_component_of_wind           (time, level, longitude, latitude) float32 10GB dask.array<chunksize=(100, 13, 64, 32), meta=np.ndarray>
    vertical_velocity             (time, level, longitude, latitude) float32 10GB dask.array<chunksize=(100, 13, 64, 32), meta=np.ndarray>

In [5]:
#rename obs data:
obs_data = obs_data.rename({'longitude': 'lon'})
obs_data = obs_data.rename({'latitude': 'lat'})

# add datetime coord:
copied_coord = obs_data['time'].copy()
# Rename the copied coordinate to the new name
obs_data = obs_data.assign_coords(datetime=copied_coord)

# add batch dim:
obs_data = obs_data.expand_dims('batch')

In [8]:
# save obs_data
obs_data.to_zarr(absolute_path, mode='w')